In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image
from transformers import pipeline


# -----------------------------
# Load Models (Assume models are uploaded in Kaggle)
# -----------------------------
stage1_model = tf.keras.models.load_model("/kaggle/input/models-for-project/Bone_Chest_Brain_Classifier_model.keras")
bone_model   = tf.keras.models.load_model("/kaggle/input/models-for-project/Bone_x_ray_img_Detection_model.keras")
chest_model  = tf.keras.models.load_model("/kaggle/input/models-for-project/Pneumonia_TB_X_Ray_Img_Detection_model.keras")
brain_model  = tf.keras.models.load_model("/kaggle/input/models-for-project/Brain_X_ray_Img_Detection_model.keras")

# Helper Functions

def preprocess_image(img, target_size=(96,96)):
    img = img.resize(target_size)
    img_array = np.array(img).astype("float32")   # keep 0–255 values
    img_array = np.expand_dims(img_array, axis=0)
    return img_array
 

generator = pipeline("text-generation", model="gpt2")

def get_patient_suggestions(disease_name):
    prompt = f"Suggest precautions for a patient with {disease_name}"
    result = generator(prompt, max_new_tokens=100, truncation=True, num_return_sequences=1)
    return result[0]['generated_text']

def predict_stage1_stage2(image_path_or_pil):
    """
    Predict Stage 1 (image type) and Stage 2 (problem) for a single image.
    Returns a dictionary with predicted class and confidence for both stages.
    """
    # Load image
    if isinstance(image_path_or_pil, str):
        image = Image.open(image_path_or_pil).convert("RGB")
    else:
        image = image_path_or_pil.convert("RGB")
    
    # Preprocess
    img_array = preprocess_image(image)

    # -------- Stage 1 Prediction --------
    stage1_classes = ["Bone", "Chest", "Brain"]
    stage1_preds = stage1_model.predict(img_array)
    stage1_idx = np.argmax(stage1_preds, axis=1)[0]
    stage1_class = stage1_classes[stage1_idx]

    stage1_conf  = stage1_preds[0][stage1_idx]

    # -------- Stage 2 Prediction --------
    if stage1_class == "Bone":
        stage2_model = bone_model
        stage2_classes = ['Normal Bone', 'Osteoarthritis', 'Osteoporosis']
    elif stage1_class == "Chest":
        stage2_model = chest_model
        stage2_classes = ["Pneumonia", "Tuberculosis", "Normal"]
    else:
        stage2_model = brain_model
        stage2_classes = ['Aneurysm', 'Cancer', 'Tumor','Normal']

    stage2_preds = stage2_model.predict(img_array)
    stage2_idx = np.argmax(stage2_preds, axis=1)[0]
    stage2_class = stage2_classes[stage2_idx]
    stage2_conf  = stage2_preds[0][stage2_idx]

    # -------- Return Results --------
    print("This is an AI-generated output for testing purposes only.\n")
        
    print(f"X-Ray Image of : {stage1_class} Detected < Confidence {stage1_conf}>")
    print(f'Model\'s Findout: {stage2_class} < Confidence {stage2_conf}> ')

    
    # Load a small LLM locally
    if stage2_class == "Normal" or stage2_class == "Normal Bone":
        print("Model Predicts all good")
    else:
        print(get_patient_suggestions(stage2_class))
    return "\n Thanks for using our model"

Device set to use cpu


In [5]:
result = predict_stage1_stage2("/kaggle/input/brain-x-ray-images/test/Tumor/1_8.jpg")
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 900ms/step


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is an AI-generated output for testing purposes only.

X-Ray Image of : Brain Detected < Confidence 0.9960758090019226>
Model's Findout: Tumor < Confidence 0.9984140396118164> 
Suggest precautions for a patient with Tumor:

Take at least one dose of anti-inflammatory medication every day, including aspirin, ibuprofen, or epinephrine.

Avoid taking blood thinners (blood thinner tablets) if they are not prescribed.

Do not take any medications in the morning.

Have a good sleep.

Avoid the use of painkillers and alcohol at night.

Avoid excessive use of certain medications.

Use at night.

Keep a full schedule of

 Thanks for using our model
